In [1]:
import os
import cv2
import math
import time
import pickle
import numpy as np
from random import randint
from skimage.feature import hog
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.svm import LinearSVC, SVC
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [2]:
Veh_Imgs_Path = "./../udacity_dataset/vehicles/vehicles/"
NonVeh_Imgs_Path = "./../udacity_dataset/non_vehicles/non_vehicles/"

Veh_Imgs = os.listdir(Veh_Imgs_Path)
NonVeh_Imgs = os.listdir(NonVeh_Imgs_Path)

In [3]:
def CollectImages(folderpath, ImagePaths, isLabeled = True):
    images = []
    for ImagePath in ImagePaths:
        if (len(ImagePath.split('.')) == 2):
            im = cv2.imread(folderpath + ImagePath)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
            #print('Image ' , ImagePath , ' is: ', type(im), ' with dimensions: ', im.shape)
            if isLabeled == True:
                images.append((im, ImagePath.split('.')[0]))
            else:
                images.append(im)
        else:
            print('Image', ImagePath, 'is correupted --> Discarded.')

    return images

In [4]:
CarImageSet = []
NonCarImageSet = []

for folder in Veh_Imgs:
    folderpath = Veh_Imgs_Path + folder
    Carsimagespath = os.listdir(folderpath)
    CarImageSet.append(CollectImages(folderpath + '/', Carsimagespath, True))

for folder in NonVeh_Imgs:
    folderpath = NonVeh_Imgs_Path + folder
    NonCarsimagespath = os.listdir(folderpath)
    NonCarImageSet.append(CollectImages(folderpath + '/', NonCarsimagespath, True))

CarImageSet = np.concatenate(CarImageSet)
NonCarImageSet = np.concatenate(NonCarImageSet)

print('Car Image Set has ', str(len(CarImageSet)))
print('Non Car Image Set has ', str(len(NonCarImageSet)))

Car Image Set has  8792
Non Car Image Set has  8968


In [5]:
imgPxlVal = 0
img_Label = 1

In [6]:
def pltImages(images, labels, nrows = 1, ncols = 2, fig_w = 20, fig_h = 10, isgray = False):
    #below code is inspired from https://stackoverflow.com/questions/17111525/how-to-show-multiple-images-in-one-figure
    assert len(images) <= (nrows * ncols)
    if labels != "":
        assert len(images) == len(labels)
    
    fig = plt.figure(figsize=(fig_w, fig_h))
    
    for index in range(len(images)):
        plot = fig.add_subplot(nrows,ncols,index+1)
        if labels != "":
            plot.set_title(labels[index])
        if(isgray == False):
            plt.imshow(images[index].squeeze())
        else:
            plt.imshow(images[index].squeeze(), cmap='gray')
    
    plt.tight_layout()
    plt.show()

In [7]:
#Code obtained from Udacity lesson 16
def bin_spatial_Demo_colorspace(img, color_space='RGB', size=(32, 32)):
    # Convert image to new color space (if specified)
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'LAB':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else: feature_image = np.copy(img)             
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(feature_image, size).ravel() 
    # Return the feature vector
    return features

In [8]:
def get_hog_features_demo(img, orient, pix_per_cell, cell_per_block, blockNorm, trns_sqrt, visualize=False, feature_vec=True):
    if visualize == True:
        features, hog_im = hog(img,
                               orientations=orient,
                               pixels_per_cell=(pix_per_cell, pix_per_cell),
                               cells_per_block=(cell_per_block, cell_per_block),
                               block_norm = blockNorm, #Block Normalize didn't show any change effect on image
                               transform_sqrt=trns_sqrt, 
                               visualise=visualize,
                               feature_vector=feature_vec)
        return features, hog_im
    else:      
        features = hog(img,
                       orientations=orient,
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block),
                       block_norm = blockNorm, #Block Normalize didn't show any change effect on image
                       transform_sqrt=trns_sqrt, 
                       visualise=visualize,
                       feature_vector=feature_vec)
        return features

In [9]:
def normalize(data):
    return preprocessing.normalize(data)

In [10]:
def extractFeatures(img,debug=False,ReturnSizes=False):
    
    hist_feat = []
    SB_feat = []
    hog_feat = []
    
    hist_feat_shape = 0
    SB_feat_shape   = 0
    hog_feat_shape  = 0
    
    histrange = [0,255]
    
    if CHist_EN == True:
        #Extract Color Hitogram Features
        if Hist_colorspace == "HSV":
            hist_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif Hist_colorspace == "HLS":
            hist_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif Hist_colorspace == "LAB":
            hist_image = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        elif Hist_colorspace == "YUV":
            hist_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif Hist_colorspace == "LUV":
            hist_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif Hist_colorspace == "YCrCb":
            hist_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else: #Assume No action needed
            hist_image = np.copy(img)
        
        if CS_Ch_Idx != "All":
            hist_feat = cv2.calcHist(hist_image,[CS_Ch_Idx],None,histSize,histrange)
        else:
            hist_feat = []
            for Ch_Idx in range(hist_image.shape[2]):
                hist_feat.append(cv2.calcHist(hist_image,[Ch_Idx],None,histSize,histrange))
        
        hist_feat = np.ravel(hist_feat).reshape(1, -1)

        if debug == True:
            print("Histogram")
            print("Before Normalize")
            print(hist_feat.shape)
            print(hist_feat)
        
        if Nrmlz == True:
            hist_feat = normalize(hist_feat)
            if debug == True:
                print("After Normalize")
                print(hist_feat.shape)
                print(hist_feat)
                
        hist_feat = hist_feat[0]
        hist_feat_shape = hist_feat.shape
        
    if SB_EN == True:
        #Extract Spatial Binning Features
        if Hist_colorspace == "HSV":
            SB_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif Hist_colorspace == "HLS":
            SB_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif Hist_colorspace == "LAB":
            SB_image = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        elif Hist_colorspace == "YUV":
            SB_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif Hist_colorspace == "LUV":
            SB_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif Hist_colorspace == "YCrCb":
            SB_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else: #Assume No action needed
            SB_image = np.copy(img)
        
        SB_feat = cv2.resize(SB_image, size).ravel().reshape(1, -1).astype(np.float64)

        if debug == True:
            print("Spatial Binning")
            print("Before Normalize")
            print(SB_feat.shape)
            print(SB_feat)
        
        if Nrmlz == True:
            SB_feat = normalize(SB_feat)
            if debug == True:
                print("After Normalize")
                print(SB_feat.shape)
                print(SB_feat)
                
        SB_feat = SB_feat[0]
        SB_feat_shape = SB_feat.shape
    
    if HOG_EN == True:
        #Extract HOG Features
        if HOG_colorspace == "HSV":
            hog_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif HOG_colorspace == "HLS":
            hog_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif HOG_colorspace == "LAB":
            hog_image = cv2.cvtColor(img, cv2.COLOR_RGB2LAB)
        elif HOG_colorspace == "YUV":
            hog_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif HOG_colorspace == "LUV":
            hog_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif HOG_colorspace == "YCrCb":
            hog_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
        else: #Assume No action needed
            hog_image = np.copy(img)
        
        if HOG_Ch_Idx != "All":
            hog_feat = get_hog_features_demo(
                img[:,:,HOG_Ch_Idx],
                orient,
                pix_per_cell,
                cells_per_block,
                blk_Norm,
                visualize=False,
                trns_sqrt=TransformSqrt,
                feature_vec=True)
        else:
            for Ch_Idx in range(hog_image.shape[2]):
                hog_feat.append(get_hog_features_demo(
                    img[:,:,Ch_Idx],
                    orient,
                    pix_per_cell,
                    cells_per_block,
                    blk_Norm,
                    visualize=False,
                    trns_sqrt=TransformSqrt,
                    feature_vec=True))
        
        hog_feat = np.ravel(hog_feat).reshape(1, -1)
        
        if debug == True:
            print("HOG")
            print("Before Normalize")
            print(hog_feat.shape)
            print(hog_feat)
        
        if Nrmlz == True:
            hog_feat = normalize(hog_feat)
            if debug == True:
                print("After Normalize")
                print(hog_feat.shape)
                print(hog_feat)
                
        hog_feat = hog_feat[0]
        hog_feat_shape = hog_feat.shape
    
    if debug == True:
        print("Before Concatenation")
        print(hist_feat)
        print(SB_feat)
        print(hog_feat)
    
    features = np.hstack((hist_feat, SB_feat, hog_feat))
    
    if debug == True:
        print("Output")
        print(len(features))
        print(features)
    
    #features = np.ravel(features)
    
    #if debug == True:
        #print(features.shape)
        #print(features)

    if ReturnSizes == False:
        return features
    else:
        return (hist_feat_shape, 
                SB_feat_shape, 
                hog_feat_shape, 
                features)

In [11]:
def TrainLinearSVCModel(feat, label):
    svc = LinearSVC()
    svc.fit(feat, label)

    return svc

def TrainModel_WithParameterSearch(feat, label):
    parameters = {'C':[0.1, 100]}

    svc = LinearSVC()
    clf = GridSearchCV(svc, parameters)
    clf.fit(feat, label)

    return clf

def TrainSVCModel(feat, label):
    svc = SVC(kernel='linear')
    svc.fit(feat, label)

    return svc

#below code is obtained from http://scikit-learn.org/stable/modules/model_persistence.html and twicked to dumb dictionary data
def SaveClassifier(clf, Scaler, CHist_EN, SB_EN, HOG_EN, Nrmlz,
                   histSize, Hist_colorspace, CS_Ch_Idx, size, 
                   SB_colorspace, HOG_colorspace, orient, pix_per_cell,
                   cells_per_block, blk_Norm, TransformSqrt, 
                   HOG_Ch_Idx, Name='TrainedClassifier.pkl'):
    
    Class_dict = { 'clf': clf, 'Scaler': Scaler, 'CHist_EN' : CHist_EN, 'SB_EN' : SB_EN, 'HOG_EN' : HOG_EN,
                   'Nrmlz' : Nrmlz, 'histSize' : histSize, 'Hist_colorspace' : Hist_colorspace,
                   'CS_Ch_Idx' : CS_Ch_Idx, 'size' : size, 'SB_colorspace' : SB_colorspace,
                   'HOG_colorspace' : HOG_colorspace, 'orient' : orient, 'pix_per_cell' : pix_per_cell,
                   'cells_per_block' : cells_per_block, 'blk_Norm' : blk_Norm, 'TransformSqrt' : TransformSqrt,
                   'HOG_Ch_Idx' : HOG_Ch_Idx}
    
    output = open(Name, 'wb')
    pickle.dump(Class_dict, output)
    output.close()

In [12]:
PTNormalize      = 0
PTCHist_EN       = 1
PTCHist_BCnt     = 2
PTCHist_CSpace   = 3
PTCHist_Ch_Idx   = 4
PTSB_EN          = 5
PTSB_Size        = 6
PTSB_CSpace      = 7
PTHOG_EN         = 8
PTHOG_OrientCnt  = 9
PTHOG_Px_Per_Cl  = 10
PTHOG_Cl_Per_Blk = 11
PTHOG_Blk_Nrmlz  = 12
PTHOG_Trnsf_Sqrt = 13
PTHOG_CSpace     = 14
PTHOG_Ch_Indx    = 15

RepeatCount = 1 #To Ensure the Accuracy despite the Random state

In [13]:
def ParameterTuning(SaveModel = False):
    global CHist_EN
    global SB_EN
    global HOG_EN
    global Nrmlz
    global histSize
    global Hist_colorspace
    global CS_Ch_Idx
    global size
    global SB_colorspace
    global HOG_colorspace
    global orient
    global pix_per_cell
    global cells_per_block
    global blk_Norm
    global TransformSqrt
    global HOG_Ch_Idx
     
    SearchTrials=[
        [True ,True ,[64],"LUV","All",True ,(16,16),"LUV",True ,8,16,2,"L2",True ,"LUV","All"]
    ]

    for cnt in range(0,RepeatCount):
        for idx in range(0, len(SearchTrials)):
            CHist_EN  = SearchTrials[idx][PTCHist_EN]
            SB_EN     = SearchTrials[idx][PTSB_EN]
            HOG_EN    = SearchTrials[idx][PTHOG_EN]
            Nrmlz     = SearchTrials[idx][PTNormalize]
            histSize  = SearchTrials[idx][PTCHist_BCnt]
            Hist_colorspace = SearchTrials[idx][PTCHist_CSpace]
            CS_Ch_Idx = SearchTrials[idx][PTCHist_Ch_Idx]
            size      = SearchTrials[idx][PTSB_Size]
            SB_colorspace   = SearchTrials[idx][PTSB_CSpace]
            HOG_colorspace  = SearchTrials[idx][PTHOG_CSpace]
            orient    = SearchTrials[idx][PTHOG_OrientCnt]
            pix_per_cell    = SearchTrials[idx][PTHOG_Px_Per_Cl]
            cells_per_block = SearchTrials[idx][PTHOG_Cl_Per_Blk]
            blk_Norm  = SearchTrials[idx][PTHOG_Blk_Nrmlz]
            TransformSqrt   = SearchTrials[idx][PTHOG_Trnsf_Sqrt]
            HOG_Ch_Idx= SearchTrials[idx][PTHOG_Ch_Indx]

            #Test = extractFeatures(HOGCarImage[0])

            #start_time = time.time()
            
            lCar_features = []
            lNonCar_features = []

            for image in CarImageSet:
                COut = extractFeatures(image[imgPxlVal], ReturnSizes=True)
                lCar_features.append(COut[3])

            for image in NonCarImageSet:
                NCOut = extractFeatures(image[imgPxlVal], ReturnSizes=True)
                lNonCar_features.append(NCOut[3])

            lFeatureSet = np.vstack((lCar_features, lNonCar_features)).astype(np.float64)
            lLbaelSet = np.hstack((np.ones(len(lCar_features)), np.zeros(len(lNonCar_features))))

            lrand_state = 20
            #lrand_state = np.random.randint(0, 100)
            lFeature_train, lFeature_test, lLabel_train, lLabel_test = train_test_split(lFeatureSet, lLbaelSet, test_size=0.2, random_state=lrand_state)
            
            lFeature_scaler = StandardScaler().fit(lFeature_train)
            lScaled_FeatureTrainSet = lFeature_scaler.transform(lFeature_train)
            lScaled_FeatureTestSet = lFeature_scaler.transform(lFeature_test)

            clf = TrainLinearSVCModel(lScaled_FeatureTrainSet, lLabel_train)
            #clf = TrainModel_WithParameterSearch(lFeature_train, lLabel_train)
            
            if SaveModel == True:
                Name = "TrainedClassifier" + str(cnt) + ".pkl"
                SaveClassifier(clf, lFeature_scaler, CHist_EN, SB_EN, HOG_EN, Nrmlz, 
                               histSize, Hist_colorspace, CS_Ch_Idx, size, SB_colorspace, 
                               HOG_colorspace, orient, pix_per_cell, cells_per_block, blk_Norm, TransformSqrt,
                               HOG_Ch_Idx, Name)

            start_time = time.time()
            
            acc = clf.score(lScaled_FeatureTestSet, lLabel_test)
            
            elapsed_time = time.time() - start_time

            print("Param Set(",idx,") result in acc of ",acc," CH Feat cnt=",COut[0]," SB Feat cnt=",COut[1]," HOG Feat cnt=",COut[2]," In time:",elapsed_time)
            #print("Param value", clf.best_params_)
    
ParameterTuning(True)

Param Set( 0 ) result in acc of  0.990990990991  CH Feat cnt= (192,)  SB Feat cnt= (768,)  HOG Feat cnt= (864,)  In time: 0.02031254768371582
